# EXERCISE - 1: PRINTING OUTPUT DISTRIBUTION AND MAKING IT UNIFORM

In [1]:
import pandas as pd
from langchain_community.llms import Ollama

file_path = 'C:/Users/Ria/OneDrive/Desktop/PESU/UE21CS326C - Large Language Models/Day 1/train.csv'
df = pd.read_csv(file_path)

distribution = df['is_duplicate'].value_counts(normalize=True) * 100
print("Initial distribution:\n", distribution)

llama = Ollama(model="llama3.1")

Initial distribution:
 is_duplicate
0    63.080215
1    36.919785
Name: proportion, dtype: float64


In [3]:
null_data = df[df.isnull().any(axis=1)]
null_data

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [9]:
ollama = Ollama(model="llama3.1")

def generate_data(prompt):
    response = ollama(prompt)
    return response

def fill_null_with_llm(df, column_name, prompt_template):
    for index, row in df[df[column_name].isnull()].iterrows():
        prompt = prompt_template.format(row=row)
        generated_data = generate_data(prompt)
        df.at[index, column_name] = generated_data
    return df

prompt_template = "Generate a suitable value for the following context: {row}"

df = fill_null_with_llm(df, 'question1', prompt_template)
df = fill_null_with_llm(df, 'question2', prompt_template)

print("Null values after filling:")
print(df.isnull().sum())

# saving the DataFrame to a new CSV file
df.to_csv('C:/Users/Ria/OneDrive/Desktop/PESU/UE21CS326C - Large Language Models/Day 1/train2.csv', index=False)

Null values before filling:
id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64
Null values after filling:
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


In [2]:
df = pd.read_csv('C:/Users/Ria/OneDrive/Desktop/PESU/UE21CS326C - Large Language Models/Day 1/train2.csv')

In [3]:
def generate_synthetic_question(original_question):
    # generating a synthetic question based on the original question
    prompt = f"Generate a synthetic question similar to: {original_question}. Return ONLY the generated question and nothing else."
    response = llama(prompt)
    return response

In [4]:
# function to balance the dataset if it is imbalanced
def balance_dataset(df):
    distribution = df['is_duplicate'].value_counts()
    majority_class = distribution.idxmax()
    minority_class = distribution.idxmin()

    if distribution[0] == distribution[1]:
        return df

    num_samples_needed = abs(distribution[0] - distribution[1])
    minority_samples = df[df['is_duplicate'] == minority_class]
    
    synthetic_samples = []
    for _, row in minority_samples.sample(num_samples_needed, replace=True).iterrows():
        synthetic_question1 = generate_synthetic_question(row['question1'])
        synthetic_question2 = generate_synthetic_question(row['question2'])
        print(synthetic_question1)
        print(synthetic_question2)
        synthetic_samples.append([row['id'], row['qid1'], row['qid2'], synthetic_question1, synthetic_question2, minority_class])

    synthetic_df = pd.DataFrame(synthetic_samples, columns=df.columns)

    balanced_df = pd.concat([df, synthetic_df])
    return balanced_df

In [5]:
balanced_df = balance_dataset(df)

C:\Users\Ria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


How can I transition into a data science role with experience in computer programming from a mechanical engineering background?
What are the most important programming languages to learn for a career transition from ECE to data science?
What is the most thought-provoking short story that has left an indelible mark on your perspective?
What is the most mesmerizing music festival you would attend if you had unlimited resources?
What strategies would you employ to excel in virtual job fairs?
What strategies can an individual employ to effectively manage their time during a job search, particularly when juggling multiple applications and deadlines?
Can the United States enhance its renewable energy production more efficiently than Germany within five years by investing in local research and development rather than relying on international imports?
Can Singapore surpass South Korea in renewable energy innovation by 2035?
What is the main difference between cooked sausages and roasted poultr

KeyboardInterrupt: 

In [ ]:
# check the new distribution
new_distribution = balanced_df['is_duplicate'].value_counts(normalize=True) * 100
print("New distribution:\n", new_distribution)